# Climate Change India NLP
The primary goal of this script is to analyze the sentiment of climate change news articles published in India from 2013- 2016. To do this, I will first use a data set from Jogesh 2012, that has manually coded sentiments to train my model. I will then fit this model to the new data.

1. Data clean up
2. Use weblinks to get newsarticle content and text using the newspaper library
3. Use NLP to clean the text data 
4. Use machine learning to associate words to sentiments in training data
5. Apply this to test data 

In [69]:
#The data weblinks first need to extracted from excel files which I did in R -- 
#no idea how to read excel files in python!. 

import pandas as pd
df = pd.read_csv('cleanURLData.csv')
df.columns = ['num','num2','url','codes']

print(df.head())
#Write a function to replace codes with sentiments

#make the separators consistant + to , and remove spaces
df['codes'] = df['codes'].replace(['\+'],', ', regex=True)
df['codes'] = df['codes'].replace([' '],'', regex=True)

#split comma separated codes into individual categories
splitcodes=df['codes'].str.split(',', expand=True)

print(splitcodes.head())

#create new df 
sentiment = splitcodes

#change codes to real sentiments 
#obj =splitcodes[0].replace(['^S'], 'Science', regex=True)
sentiment=splitcodes.replace(['^S'], 'Science', regex=True)
sentiment=sentiment.replace(['^GP'], 'GlobalPolitics', regex=True)
sentiment=sentiment.replace(['^IS'], 'IndianScience', regex=True)
sentiment=sentiment.replace(['^IP'], 'IndianPolitics', regex=True)
sentiment=sentiment.replace(['^BZ'], 'Buisness', regex=True)
sentiment=sentiment.replace(['^TD'], 'Trends', regex=True)

#append sentiment to main df
df['sentiment']=sentiment[0]
print(df.head())



   num  num2                                                url          codes
0    1   NaN  http://www.indianexpress.com/news/the-green-co...            NaN
1    2   2.0  http://www.indianexpress.com/news/china-and-in...  S, A6, D1, T1
2    3   3.0  http://www.financialexpress.com/news/climate-t...      S, A1, T7
3    4   4.0  http://www.livemint.com/2009/08/31211415/Tech-...     GP, A6, T2
4    5   5.0  http://lite.epaper.timesofindia.com/getpage.as...  S, A6, D1, T6
     0     1     2     3     4     5     6     7
0  NaN  None  None  None  None  None  None  None
1    S    A6    D1    T1  None  None  None  None
2    S    A1    T7  None  None  None  None  None
3   GP    A6    T2  None  None  None  None  None
4    S    A6    D1    T6  None  None  None  None
   num  num2                                                url       codes  \
0    1   NaN  http://www.indianexpress.com/news/the-green-co...         NaN   
1    2   2.0  http://www.indianexpress.com/news/china-and-in...  S,A6,D1,T

## Extract Newspaper data from html links


In [ ]:
import warnings; warnings.simplefilter('ignore') #need to do this for a few invalid links in the url list
from newspaper import Article	

def html_to_text( urls ):
		a = Article(urls) 
		a.download()
		a.parse()
		a.nlp()
		text = a.text
		title = a.title
		authors = a.authors
		date = a.publish_date
		sourceurl = a.source_url
		keywords = a.keywords
		data =(text,title,sourceurl,authors,date,keywords)
		return(data)

html_to_text(df['url'])
Alldata =[]
for url in df['url']:
	try:
		Alldata.append(html_to_text(url))
	except:
		pass
#will return a list of tuples with all the data. 

newsdf = pd.DataFrame(Alldata)
pd.concat(Alldata).to_csv('Alldata.csv')
newsdf.columns = ['text', 'title','source','author','date','keywords']
print(newsdf.columns.values)


Traceback (most recent call last):
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseRes

You must download and parse an article before parsing it!


Traceback (most recent call last):
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseRes

You must download and parse an article before parsing it!


Traceback (most recent call last):
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseRes

...

You must download and parse an article before parsing it!


Traceback (most recent call last):
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseRes

You must download and parse an article before parsing it!


Traceback (most recent call last):
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/Users/taniajogesh/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582, in lxml.etree._ParserContext._handleParseRes

You must download and parse an article before parsing it!


In [ ]:
newsdf = pd.DataFrame(Alldata)
pd.concat(Alldata).to_csv('Alldata.csv')
newsdf.columns = ['text', 'title','source','author','date','keywords']
print(newsdf.head())

